# Assignment 2 Task 2
## Pulp Fiction Movie Reviews Scraping

In [1]:
from bs4 import BeautifulSoup
import requests

In [22]:
finalRating,finalTitle,finalReview,finalProfile,finalDate=[],[],[],[],[]
for x in range(1,5):
  for i in range(1,11):
    url="https://www.imdb.com/title/tt0110912/reviews?sort=curated&dir=desc&ratingFilter="+str(i)
    page=requests.get(url)
    if page.status_code==200:
      soup=BeautifulSoup(page.content,'html.parser')
      ratings=soup.find_all('span',class_='rating-other-user-rating')
      titles=soup.find_all('a',class_='title')
      reviews=soup.find_all('div',class_='text show-more__control')
      profiles=soup.find_all('span',class_='display-name-link')
      dates=soup.find_all('span',class_='review-date')
      for rating in ratings:
        finalRating.append(int(rating.text.strip().replace('/10','')))
      for title in titles:
        finalTitle.append(title.text.strip())
      for review in reviews:
        finalReview.append(review.text.strip())
      for profile in profiles:
        finalProfile.append(profile.text.strip())
      for date in dates:
        finalDate.append(date.text.strip())
    else:
      print(f"<Respponse {page.status_code}>")



In [24]:
print(len(finalRating),len(finalTitle),len(finalReview),len(finalProfile),len(finalDate))

250 250 250 250 250


In [25]:
#creating a dataframe out of obtained data
import pandas as pd
df=pd.DataFrame({'Rating':finalRating,'Review title':finalTitle,'Review body':finalReview,'Username':finalProfile,'Date of review':finalDate})

In [26]:
#shuffle the dataframe entries
df=df.sample(frac=1)

In [27]:
df

,Rating,Review title,Review body,Username,Date of review
222,9,"One of the most impactful, entertaining and me...",Pulp fiction is a type of movie which makes a ...,jashminocha,21 November 2019
243,10,This is Why Tarantino is my favorite,One of my favorite movies of all time and This...,logo-48974,15 July 2020
96,4,Most Overrated Movie Ever,It's definately overrated. Not something speci...,dimitrispappas-71186,17 November 2018
73,3,Aged Poorly,First 1/2 is so dry I almost turned it off.Thi...,bianca-rose-48029,25 September 2022
165,7,"Watched it because of the rating, however didn...","Quick disclaimer, I am 24 and have generally n...",chervenkov,13 July 2021
...,...,...,...,...,...
51,3,overrated,I tried to complete it many times and i just c...,molg-18070,29 April 2024
66,3,Not all that I expected,When I was finally going to see this movie for...,babcockp,16 July 2006
33,2,One of the worst movies I've ever seen!!!,What is the big deal with this movie? The mov...,LebowskiT1000,21 February 2002
103,5,Painfully overrated,this movie is painfully overrated: every time ...,Margarida_do_Vale,23 October 2016


# Perform sentiment analysis using textblob
## Firstly pre-process the data

In [29]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [31]:
df['cleaned_reviews'] = df['Review body'].apply(preprocess_text)

In [39]:
#perform sentiment analysis using textblob
from textblob import TextBlob
def get_sentiment(text):
    analysis = TextBlob(text)
    Polarity=analysis.sentiment.polarity
    if Polarity > 0:
        return 'positive'
    elif Polarity < 0:
        return 'negative'
    else:
        return 'neutral'
def get_polarity(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [40]:
#create a column for polarity
df['Polarity'] = df['cleaned_reviews'].apply(get_polarity)

In [41]:
df['sentiment'] = df['cleaned_reviews'].apply(get_sentiment)

In [42]:
df

,Rating,Review title,Review body,Username,Date of review,cleaned_reviews,sentiment,Polarity
222,9,"One of the most impactful, entertaining and me...",Pulp fiction is a type of movie which makes a ...,jashminocha,21 November 2019,pulp fiction type movie makes strong argument ...,positive,0.476389
243,10,This is Why Tarantino is my favorite,One of my favorite movies of all time and This...,logo-48974,15 July 2020,one favorite movies time one essential viewing...,positive,0.411259
96,4,Most Overrated Movie Ever,It's definately overrated. Not something speci...,dimitrispappas-71186,17 November 2018,definately overrated something special idiot d...,negative,-0.140260
73,3,Aged Poorly,First 1/2 is so dry I almost turned it off.Thi...,bianca-rose-48029,25 September 2022,first dry almost turned offthis movie glorifie...,positive,0.030303
165,7,"Watched it because of the rating, however didn...","Quick disclaimer, I am 24 and have generally n...",chervenkov,13 July 2021,quick disclaimer generally seen tons movies re...,negative,-0.000238
...,...,...,...,...,...,...,...,...
51,3,overrated,I tried to complete it many times and i just c...,molg-18070,29 April 2024,tried complete many times cant boring silence ...,negative,-0.010000
66,3,Not all that I expected,When I was finally going to see this movie for...,babcockp,16 July 2006,finally going see movie first time really exci...,positive,0.155833
33,2,One of the worst movies I've ever seen!!!,What is the big deal with this movie? The mov...,LebowskiT1000,21 February 2002,big deal movie movie boring uninteresting full...,positive,0.023124
103,5,Painfully overrated,this movie is painfully overrated: every time ...,Margarida_do_Vale,23 October 2016,movie painfully overrated every time someone t...,positive,0.143452
